1. For this problem, you will simulate a two-class classification with class imbalance based on two input featuresand 10k observations, where the class of “1” is only 1% of the whole data, as shown in class. You will create your own KNearestNeighbors function based on the Usearch library. You will compare the performance of the KNN model created with cross-validation tunning for the optimal number of neighbors and the different methods for compensating the class imbalance, such as SMOTE (synthetic minority oversampling), ADASYN (adaptive synthetic minority oversampling), and FastKDE sampling.


In [10]:
# computational libraries
import numpy as np
import pandas as pd
import xgboost
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler, QuantileTransformer, MinMaxScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from scipy.spatial import Delaunay
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
import scipy.stats as stats
from sklearn.model_selection import train_test_split as tts, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error as mse
from scipy.spatial.distance import cdist
from scipy.interpolate import interp1d, RegularGridInterpolator, griddata, LinearNDInterpolator, NearestNDInterpolator
from math import ceil
from scipy import linalg
# the following line(s) are necessary if you want to make SKlearn compliant functions
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
import warnings
warnings.filterwarnings('ignore')

from sklearn import linear_model
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression

In [11]:
!pip install usearch

In [12]:
from usearch.index import Index, MetricKind, search

class KNNRegressor:
    def __init__(self, k, metric):
        self.k = k
        self.metric = metric
        self.index = None

    def fit(self, x, y):
        self.x_train = x
        self.y_train = y
        self.index = Index(ndim=x.shape[1], metric=self.metric)
        self.index.add(np.arange(len(x)), x)

    def predict(self, X):
        matches_generator = (self.index.search(x.reshape(1, -1), count=self.k) for x in X)
        predictions = np.array([np.mean(self.y_train[matches.keys]) for matches in matches_generator])
        return predictions

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
data = pd.read_csv('drive/MyDrive/concrete.csv')

In [15]:
data

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


In [16]:
x = data.loc[:,'cement':'age'].values
y = data['strength'].values

In [17]:
x_train, x_test, y_train, y_test = tts(x, y, test_size=0.2, random_state=42)
knn_regressor = KNNRegressor(k=10, metric=MetricKind.Cos)
knn_regressor.fit(x_train, y_train)
y_pred = knn_regressor.predict(x_test)
print(mse(y_test, y_pred))

86.06482597087378


2. Provide a title, a reference to the data set, an abstract, and the main goal for the capstone project.


Title: Predicting Popularity in Spotify's Most Popular Songs: A Data-Driven Analysis

Reference to Data Set: The data set utilized for this project is derived from Spotify's collection of most popular songs, encompassing various attributes such as track features, artist information, and popularity metrics. It can be found here: https://www.kaggle.com/datasets/nelgiriyewithana/top-spotify-songs-2023/data

Abstract:
What makes a song a banger? Understanding the factors influencing a song's popularity is crucial for artists, record labels, and music enthusiasts alike. In this capstone project, we delve into Spotify's dataset of most popular songs to uncover trends, patterns, and predictors of song popularity. Through exploratory data analysis and machine learning techniques, we aim to gain insights into what makes a song successful on the platform. By developing predictive models, we strive to forecast the potential popularity of new and upcoming songs.

Main Goal:
There are two main goals of this project. We seek to uncover key attributes and trends that drive the popularity of songs on Spotify as well as to develop predictive models capable of forecasting the popularity of new songs.